# Metadata
```
Author: Alice Bogdan (ucn3qn@virginia.edu)
Course: DS 5001
Date: 3 May 2022
Semester Project
Topic: Gilmore Girls
```

Purpose: Using ETA on Gilmore Girls tv scripts. Generate tfidf, VOCAB, and BOW using Sklearn

# Set Up

In [1]:
#libraries
import pandas as pd
import numpy as np
import nltk
# from sklearn.decomposition import PCA
from scipy.linalg import norm
import plotly.express as px
import seaborn as sns; sns.set()
import re
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from gensim.models import word2vec
from sklearn.manifold import TSNE
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer #either just vectorizer or count/transformer in tandem

In [2]:
OHCO = ['Season','Episode','line_num','Character','sent_num','token_num']

In [3]:
#for convenience
SEAS = OHCO[:1]
EPI = OHCO[:2]
LINE = OHCO[:3]
CHAR = OHCO[:4]
SENT = OHCO[:5]
TOKEN = OHCO[:6]

In [4]:
#gradient colormap 
gradient_cmap = 'YlGnBu'

In [5]:
import os
import sys

In [6]:
#import libraries in local directory
sys.path.append('lib')
import hw07

In [74]:
ngram_range = (1,1) #countvectorize - pull out n-grams. Part of token list
n_terms = 4000 #subset vocab
pos_list = "NN NNS".split() #nouns only

# Load Data 

In [75]:
data_home = "data"
data_prefix = 'Gilmore_Girls'

In [76]:
#main data sets
DATA = pd.read_csv(f"{data_home}/output/{data_prefix}-DATA.csv")
#CORPUS = pd.read_csv(f"{data_home}/output/{data_prefix}-TOKENS.csv") #aka the CORPUS
#VOCAB = pd.read_csv(f"{data_home}/output/{data_prefix}-VOCAB.csv")
VOCAB2 = pd.read_csv(f"{data_home}/output/{data_prefix}-VOCAB.csv").set_index('term_str')
CORPUS2 = pd.read_csv(f"{data_home}/output/{data_prefix}-CORPUS.csv").set_index(TOKEN) #aka the CORPUS

In [77]:
#libraries
LIB_SEASON = pd.read_csv(f"{data_home}/output/{data_prefix}-LIB-SEASON.csv").set_index(['Season'])
LIB_EPISODE = pd.read_csv(f"{data_home}/output/{data_prefix}-LIB-EPISODE.csv")

In [78]:
LIB_SEASON

,Show,Num_Episodes,Year,Num_Lines
Season,,,,
1,Gilmore Girls,21,2000,16031
2,Gilmore Girls,22,2001,18264
3,Gilmore Girls,22,2002,17699
4,Gilmore Girls,22,2003,17806
5,Gilmore Girls,22,2004,16737
6,Gilmore Girls,22,2005,16145
7,Gilmore Girls,22,2006,15188


In [79]:
#character info
CHAR_INFO = pd.read_csv(f"{data_home}/output/Character-Info.csv")

In [80]:
#remove first column
DATA = DATA[["Season", "Episode", "Title","time_raw","line_num","Character","lines","Year"]]

In [81]:
VOCAB2.head()

,n,n_chars,p,s,i,h,max_pos,n_pos,cat_pos,stop,stem_porter,stem_snowball,stem_lancaster,max_pos_group
term_str,,,,,,,,,,,,,,
i,53349,1,0.045899,21.786969,4.445394,0.204039,PRP,2,"{'PRP', 'NN'}",1,i,i,i,PR
you,44496,3,0.038282,26.121741,4.707179,0.180202,PRP,2,"{'PRP', 'NN'}",1,you,you,you,PR
the,31072,3,0.026733,37.407087,5.225240,0.139686,DT,2,"{'NN', 'DT'}",1,the,the,the,DT
to,26174,2,0.022519,44.407160,5.472720,0.123240,TO,2,"{'NN', 'TO'}",1,to,to,to,TO
s,25469,1,0.021912,45.636381,5.512112,0.120783,NN,1,{'NN'},1,s,s,s,NN


In [82]:
CORPUS2.head()

pos_tuple pos  \
Season Episode line_num Character sent_num token_num                         
1      1       5        LORELAI   0        0          ('Please', 'VB')  VB   
                                           1            ('Luke', 'NN')  NN   
                                  1        0          ('Please', 'VB')  VB   
                                           1          ('please', 'NN')  NN   
                                           2          ('please', 'NN')  NN   

                                                     token_str term_str  \
Season Episode line_num Character sent_num token_num                      
1      1       5        LORELAI   0        0            Please   please   
                                           1              Luke     luke   
                                  1        0            Please   please   
                                           1            please   please   
                                           2            please   please   

                                                     pos_group  
Season Episode line_num Character sent_num token_num            
1      1       5        LORELAI   0        0                VB  
                                           1                NN  
                                  1        0                VB  
                                           1                NN  
                                           2                NN

In [83]:
CHAR_INFO

,Character,Gender,Status,S1_Age,S2_Age,S3_Age,S4_Age,S5_Age,S6_Age,S7_Age
0,DEAN,M,Main,Teen,Teen,Teen,YA,YA,YA,YA
1,EMILY,F,Main,OA,OA,OA,OA,OA,OA,OA
2,MICHEL,M,Main,Adult,Adult,Adult,Adult,Adult,Adult,Adult
3,PARIS,F,Main,Teen,Teen,Teen,YA,YA,YA,YA
4,JASON,M,Main,Adult,Adult,Adult,Adult,Adult,Adult,Adult
...,...,...,...,...,...,...,...,...,...,...
89,SHERRY,F,Family,Adult,Adult,Adult,Adult,Adult,Adult,Adult
90,SASHA,F,Family,Adult,Adult,Adult,Adult,Adult,Adult,Adult
91,PAUL ANKA,M,Stars_Hollow,Adult,Adult,Adult,Adult,Adult,Adult,Adult
92,CAESAR,M,Stars_Hollow,Adult,Adult,Adult,Adult,Adult,Adult,Adult


In [84]:
LIB_EPISODE = LIB_EPISODE.reset_index()

In [85]:
LIB_EPISODE['label'] = LIB_EPISODE.apply(lambda x: f"{x.Season}x{x.Episode}–{x.Title[:10].upper()}", 1)

In [86]:
LIB_EPISODE = LIB_EPISODE.drop(columns = ['index'])
LIB_EPISODE = LIB_EPISODE.set_index(OHCO[:2])

# Create DOC

In [87]:
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

In [88]:
DOC = gather_docs(CORPUS2, 2)

In [89]:
DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))

In [90]:
DOC

doc_str  n_tokens
Season Episode                                                             
1      1        please luke please please please how many cups...      5667
       2        that s nice thank you don t move please so why...      5389
       3        dinner was lovely emily mira does make a perfe...      6123
       4        shopping for school supplies party nobody dema...      6459
       5        tomorrow our lawyer joseph stanford is coming ...      6599
...                                                           ...       ...
7      18       what are you doing what oh fine michel michel ...      8061
       19       hey well well well i always suspected this day...      8052
       20       excuse me mrs o malley i need you to remove mi...      6958
       21       how did you get roped into bringing the cake t...      7120
       22       i m telling you it s her trust me it s not it ...      7423

[153 rows x 2 columns]

# Method 1: CountVectorizer + TfidfTransformer

## Create DTM

In [91]:
count_engine = CountVectorizer(
    stop_words='english',
    ngram_range= ngram_range,
    #max_features=n_terms
)

In [92]:
X = count_engine.fit_transform(DOC.doc_str)

In [93]:
#X.toarray()

In [94]:
#count_engine.get_feature_names()

In [95]:
DTM = pd.DataFrame(X.toarray(), 
                   columns=count_engine.get_feature_names(), 
                   index=DOC.index)

In [96]:
DTM

00  000  002  005  01  03  04  05  0700  10  ...  zurich  zwi  \
Season Episode                                               ...                
1      1         0    0    0    0   0   0   0   0     0   0  ...       0    0   
       2         0    0    0    0   0   0   0   0     0   3  ...       0    0   
       3         1    1    0    0   0   0   0   0     0   0  ...       0    0   
       4         1    0    0    0   0   0   0   1     0   0  ...       0    0   
       5         3    0    0    0   0   1   0   0     0   1  ...       0    0   
...             ..  ...  ...  ...  ..  ..  ..  ..   ...  ..  ...     ...  ...   
7      18        0    1    0    0   0   0   0   0     0   0  ...       0    0   
       19        2    1    0    0   0   0   0   0     0   4  ...       0    0   
       20        1    0    0    0   0   0   0   0     0   1  ...       0    0   
       21        0    0    0    0   0   0   0   0     0   0  ...       0    0   
       22        1    0    0    0   0   0   0   0     0   0  ...       0    0   

                zydeco  zymurgy  zzz  ça  état  étoile  êtes  être  
Season Episode                                                      
1      1             0        0    0   0     0       0     0     0  
       2             0        0    0   0     0       0     0     0  
       3             0        0    0   0     0       0     0     0  
       4             0        0    0   0     0       0     0     0  
       5             0        0    0   0     0       0     0     0  
...                ...      ...  ...  ..   ...     ...   ...   ...  
7      18            0        0    0   0     0       0     0     0  
       19            0        0    0   0     0       0     0     0  
       20            0        0    0   0     0       0     0     0  
       21            0        0    0   0     0       0     0     0  
       22            0        0    0   0     0       0     0     0  

[153 rows x 25279 columns]

## Get VOCAB

In [97]:
VOCAB = DTM.sum().to_frame('n')

In [98]:
VOCAB.sort_index()

,n
00,134
000,42
002,1
005,1
01,2
...,...
ça,1
état,2
étoile,2
êtes,1


In [99]:
VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)

In [100]:
VOCAB.value_counts('n_tokens')

n_tokens
1    25279
dtype: int64

In [101]:
VOCAB

,n,n_chars,n_tokens
00,134,2,1
000,42,3,1
002,1,3,1
005,1,3,1
01,2,2,1
...,...,...,...
ça,1,2,1
état,2,4,1
étoile,2,6,1
êtes,1,4,1


In [102]:
VOCAB[VOCAB.n_tokens == 2].head(30)

,n,n_chars,n_tokens


## Create TFIDF (F4)

In [103]:
# TfidfTransformer?

In [104]:
tfidf_engine = TfidfTransformer(norm='l2', use_idf=True)

In [105]:
#tfidf_engine_smooth = TfidfTransformer(norm='l2', use_idf=True, smooth_idf = True)

In [106]:
X1 = tfidf_engine.fit_transform(DTM)

In [107]:
TFIDF = pd.DataFrame(X1.toarray(), columns=DTM.columns, index=DTM.index)

In [108]:
TFIDF

00       000  002  005   01        03   04        05  \
Season Episode                                                               
1      1        0.000000  0.000000  0.0  0.0  0.0  0.000000  0.0  0.000000   
       2        0.000000  0.000000  0.0  0.0  0.0  0.000000  0.0  0.000000   
       3        0.010142  0.012105  0.0  0.0  0.0  0.000000  0.0  0.000000   
       4        0.009546  0.000000  0.0  0.0  0.0  0.000000  0.0  0.021348   
       5        0.028135  0.000000  0.0  0.0  0.0  0.022694  0.0  0.000000   
...                  ...       ...  ...  ...  ...       ...  ...       ...   
7      18       0.000000  0.006662  0.0  0.0  0.0  0.000000  0.0  0.000000   
       19       0.015540  0.009274  0.0  0.0  0.0  0.000000  0.0  0.000000   
       20       0.009051  0.000000  0.0  0.0  0.0  0.000000  0.0  0.000000   
       21       0.000000  0.000000  0.0  0.0  0.0  0.000000  0.0  0.000000   
       22       0.008099  0.000000  0.0  0.0  0.0  0.000000  0.0  0.000000   

                0700        10  ...  zurich  zwi  zydeco  zymurgy  zzz   ça  \
Season Episode                  ...                                           
1      1         0.0  0.000000  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       2         0.0  0.032811  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       3         0.0  0.000000  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       4         0.0  0.000000  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       5         0.0  0.008468  ...     0.0  0.0     0.0      0.0  0.0  0.0   
...              ...       ...  ...     ...  ...     ...      ...  ...  ...   
7      18        0.0  0.000000  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       19        0.0  0.028062  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       20        0.0  0.008172  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       21        0.0  0.000000  ...     0.0  0.0     0.0      0.0  0.0  0.0   
       22        0.0  0.000000  ...     0.0  0.0     0.0      0.0  0.0  0.0   

                état  étoile  êtes  être  
Season Episode                            
1      1         0.0     0.0   0.0   0.0  
       2         0.0     0.0   0.0   0.0  
       3         0.0     0.0   0.0   0.0  
       4         0.0     0.0   0.0   0.0  
       5         0.0     0.0   0.0   0.0  
...              ...     ...   ...   ...  
7      18        0.0     0.0   0.0   0.0  
       19        0.0     0.0   0.0   0.0  
       20        0.0     0.0   0.0   0.0  
       21        0.0     0.0   0.0   0.0  
       22        0.0     0.0   0.0   0.0  

[153 rows x 25279 columns]

## Add stats to VOCAB

In [109]:
VOCAB['tfidf_mean'] = TFIDF.mean()
VOCAB['df'] = DTM[DTM > 0].count()
VOCAB['dfidf'] = VOCAB.df * np.log2(len(TFIDF)/VOCAB.df)

In [110]:
VOCAB.sort_values('tfidf_mean', ascending=False).head(30)

,n,n_chars,n_tokens,tfidf_mean,df,dfidf
just,7858,4,1,0.200430,153,0.000000
oh,7535,2,1,0.191832,153,0.000000
know,7417,4,1,0.189072,153,0.000000
don,6597,3,1,0.168316,153,0.000000
okay,5499,4,1,0.141075,153,0.000000
like,5100,4,1,0.129994,153,0.000000
ll,4867,2,1,0.124778,153,0.000000
yeah,4661,4,1,0.118518,153,0.000000
good,4073,4,1,0.103946,153,0.000000
right,3895,5,1,0.098994,153,0.000000


In [111]:
VOCAB.index.name = 'term_str'
VOCAB

,n,n_chars,n_tokens,tfidf_mean,df,dfidf
term_str,,,,,,
00,134,2,1,0.007643,45,79.449064
000,42,3,1,0.002762,29,69.582799
002,1,3,1,0.000139,1,7.257388
005,1,3,1,0.000139,1,7.257388
01,2,2,1,0.000254,2,12.514776
...,...,...,...,...,...,...
ça,1,2,1,0.000121,1,7.257388
état,2,4,1,0.000268,1,7.257388
étoile,2,6,1,0.000281,1,7.257388


In [112]:
VOCAB.to_csv(f'{data_home}/output/{data_prefix}-VOCAB-UNIGRAM.csv')

# Method 2: TfidfVectorizer

This method combines the two.

In [ ]:
# TfidfVectorizer?

## Create TFIDF

In [29]:
tfidf_engine2 = TfidfVectorizer(
    stop_words='english',
    ngram_range=ngram_range,
    #max_features=n_terms,
    norm='l2', 
    use_idf=True)

In [30]:
X2 = tfidf_engine2.fit_transform(DOC.doc_str)

In [31]:
TFIDF2 = pd.DataFrame(X2.toarray(), columns=tfidf_engine2.get_feature_names(), index=DTM.index)

In [ ]:
TFIDF2

## Get VOCAB

In [ ]:
VOCAB = DTM.sum().to_frame('n')

In [ ]:
VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)

In [ ]:
VOCAB['tfidf_mean'] = TFIDF2.mean()
#VOCAB = TFIDF2.mean().to_frame('tfidf_mean')

In [ ]:
VOCAB

## Add stats to VOCAB

In [ ]:
VOCAB['df'] = TFIDF2[TFIDF2 > 0].count()
VOCAB['dfidf'] = VOCAB.df * np.log2(len(TFIDF2)/VOCAB.df)

In [ ]:
VOCAB.sort_values('dfidf', ascending=False)

In [ ]:
VOCAB = VOCAB.reset_index()
VOCAB = VOCAB.rename({'index': 'term_str'}, axis=1)
VOCAB = VOCAB[['term_str','tfidf_mean','df','dfidf']]
VOCAB

In [ ]:
VOCAB2 = VOCAB2.reset_index()
VOCAB2

In [ ]:
if 'term_rank' not in VOCAB2.columns:
    VOCAB2 = VOCAB2.sort_values('n', ascending=False).reset_index()
    VOCAB2.index.name = 'term_rank'
    VOCAB2 = VOCAB2.reset_index()
    VOCAB2['term_rank'] = VOCAB2['term_rank'] + 1
    VOCAB2 = VOCAB2.set_index('term_str')

In [ ]:
new_rank = VOCAB2.n.value_counts()\
    .sort_index(ascending=False).reset_index().reset_index()\
    .rename(columns={'level_0':'term_rank2', 'index':'n', 'n':'nn'})\
    .set_index('n')

In [ ]:
new_rank

In [ ]:
VOCAB2['term_rank2'] = VOCAB2.n.map(new_rank.term_rank2) + 1

In [ ]:
VOCAB2 = VOCAB2.reset_index()
VOCAB2

In [ ]:
#VOCAB2 = VOCAB2[['term_str', 'term_rank','n','max_pos','term_rank2']]
VOCAB2 = VOCAB2[['term_str','n','max_pos']]

In [ ]:
VOCAB2

In [ ]:
VOCAB = pd.merge(VOCAB, VOCAB2, how="left", on=['term_str'])
VOCAB

In [ ]:
#VOCAB3 = VOCAB3[VOCAB3['term_str'] != "nan"]
#VOCAB3

In [ ]:
VOCAB = VOCAB.set_index('term_str')

In [ ]:
#VOCAB.to_csv(f'{data_home}/output/{data_prefix}-VOCAB-CHAR.csv')

# Create BOW

In [113]:
BOW = DTM[DTM > 0].stack().to_frame('n').join(TFIDF[TFIDF > 0].stack().to_frame('tfidf'))

In [114]:
BOW

n     tfidf
Season Episode term_str                 
1      1       200         1.0  0.017907
               314         1.0  0.028162
               absolutely  1.0  0.006541
               absurd      1.0  0.017543
               academic    1.0  0.020235
...                        ...       ...
7      22      yesterday   1.0  0.006355
               yikes       1.0  0.011749
               yo          1.0  0.010052
               ziplock     1.0  0.019597
               zipper      1.0  0.015003

[183040 rows x 2 columns]

In [115]:
BOW.to_csv(f'{data_home}/output/{data_prefix}-BOW-UNIGRAM.csv')

In [ ]:
#px.scatter(VOCAB.reset_index(), x='term_rank2', y='dfidf', color='max_pos', hover_name='term_str', height=500)

In [ ]:
#VOCAB3[VOCAB3['max_pos'] == "NN"].sample(20)

In [ ]:
BOW.sort_values('tfidf', ascending=False).head(20).style.background_gradient(cmap=gradient_cmap, high=.5)

In [ ]:
#top 1000
n_terms = 1000
pos_list = "NN NNS".split()

In [ ]:
#top 1000 terms by dfidf (no specific POS)
SIGS = VOCAB.sort_values('dfidf', ascending = False).head(n_terms).index
SIGS

In [ ]:
TFIDF2

In [ ]:
TFIDF_SIGS = TFIDF2[SIGS]
TFIDF_SIGS

In [ ]:
#subset to top 1000 nouns
VIDX = VOCAB.loc[VOCAB.max_pos.isin(pos_list)]\
    .sort_values('dfidf', ascending = False)\
    .head(n_terms).index

In [ ]:
TFIDF_N = TFIDF2[VIDX]
TFIDF_N

In [ ]:
#don't normalize mean and variance because of language (dramatically scales up the weights of rare words)
LOADINGS, DCM, COMPINF = hw07.get_pca(TFIDF_SIGS, norm_docs = True, center_by_mean = False, center_by_variance = False)

In [ ]:
#plot explained variances
COMPINF.exp_var.plot.bar(rot=0)

In [ ]:
try:
    DOC = DOC.join(DCM)
except:
    pass

In [ ]:
DOC = DOC.reset_index()

In [ ]:
DOC

In [ ]:
DOC = pd.merge(DOC, CHAR_INFO, on = 'Character')

In [ ]:
DOC = DOC.reset_index()

In [ ]:
DOC = DOC.set_index(OHCO[:2])

In [ ]:
lib_cols = "SEASON EPISODE Year Title label".split()

In [ ]:
LIB_EPISODE = LIB_EPISODE.reset_index()
LIB_EPISODE

In [ ]:
LIB_EPISODE['SEASON'] = LIB_EPISODE['Season']
LIB_EPISODE['EPISODE'] = LIB_EPISODE['Episode']
LIB_EPISODE = LIB_EPISODE.set_index(EPI)
LIB_EPISODE

In [ ]:
DOC

In [ ]:
DOC = DOC.join(LIB_EPISODE[lib_cols], on = EPI)
DOC

In [ ]:
DOC = DOC.reset_index().set_index(OHCO[:4])
DOC

In [ ]:
px.scatter(DOC, 0, 1, 
           color='SEASON', 
           symbol = 'EPISODE',
           hover_name='label', 
           marginal_x='box', marginal_y='box', 
           height=1000)

In [ ]:
COMPINF.loc[[0,5]]